## 0. Understanding Data


In [4]:
# import required modules
import sqlite3
import pandas as pd

In [5]:
# create a connection 
conn = sqlite3.connect('starwars.db')

In [6]:
# take a look at the stored tables
tables = ['people', 'films', 'starships', 'vehicles', 'species', 'planets']

for table in tables:
    display(table)
    display(pd.read_sql(f'SELECT * FROM {table} LIMIT 1', conn))

'people'

,id,name,birth_year,eye_color,gender,hair_color,height,mass,skin_color
0,1,Luke Skywalker,19BBY,blue,male,blond,172,77,fair


'films'

,id,title,episode_id,opening_crawl,director,producer,release_date
0,1,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25


'starships'

...

,id,name,model,vehicle_class,manufacturer,length,cost_in_credits,crew,passengers,max_atmosphering_speed,cargo_capacity,consumables
0,4,Sand Crawler,Digger Crawler,wheeled,Corellia Mining Corporation,36.8,150000,46,30,30,50000,2 months


'species'

,id,name,average_height,average_lifespan,classification,designation,eye_colors,hair_colors,homeworld,language,skin_colors
0,1,Human,180,120,mammal,sentient,"brown, blue, green, hazel, grey, amber","blonde, brown, black, red",https://swapi.dev/api/planets/9/,Galactic Basic,"caucasian, black, asian, hispanic"


'planets'

,id,name,diameter,rotation_period,orbital_period,gravity,population,climate,terrain,surface_water
0,1,Tatooine,10465,23,304,1 standard,200000,arid,desert,1


## 1. Required SQL analysis

### 1.1 What's the distribution of citizens across different planets
- Requirment: `1 request with INNER`
- Explanation: By using INNER JOIN, we only return people that have a known homeworld.

In [66]:
pd.read_sql(
    '''

    ''', 
    conn
)

,planet_name,people_count
0,Naboo,11
1,Tatooine,10
2,unknown,5
3,Kamino,3
4,Coruscant,3
5,Alderaan,3
6,Ryloth,2
7,Mirial,2
8,Kashyyyk,2
9,Corellia,2


### 1.2 What's the distribution of species across different planets
- Requirment: `1 request with LEFT`
- Explanation: By using LEFT JOIN, we return species regardless of whether their homeworld is known.

In [67]:
pd.read_sql(
    '''
    ''', 
    conn
)

,planet_name,species_count
0,unknown,1
1,Zolan,1
2,Vulpter,1
3,Utapau,1
4,Tund,1
5,Troiken,1
6,Trandosha,1
7,Toydaria,1
8,Tholoth,1
9,Sullust,1


### 1.3 What are the average costs, passengers, length for each vehicle class.
- Requirment: `1 request with GROUP BY`
- Explanation: By using GROUP BY, we could get the relevant values divided by vehicle class.

In [68]:
pd.read_sql(
    '''

    ''', 
    conn
)

,vehicle_class,model,average_cost,average_passengers,average_length,model_count
0,sail barge,Modified Luxury Sail Barge,285000.00,500.0,30.00,1
1,landing craft,C-9979 landing craft,200000.00,284.0,210.00,1
2,wheeled walker,"Tsmeu-6 personal wheel bike,HAVw A6 Juggernaut",182500.00,151.0,26.45,2
3,wheeled,Digger Crawler,150000.00,30.0,36.80,1
4,droid tank,NR-N99 Persuader-class droid enforcer,49000.00,4.0,10.96,1
5,repulsorcraft,"T-16 skyhopper,X-34 landspeeder,Storm IV Twin-...",35185.71,17.0,9.60,7
6,droid starfighter,tri-fighter,20000.00,0.0,5.40,1
7,air speeder,Raddaugh Gnasp fluttercraft,14750.00,0.0,7.00,1
8,airspeeder,"t-47 airspeeder,Koro-2 Exodrive airspeeder,XJ-...",14425.00,1.0,8.95,5
9,walker,"All Terrain Scout Transport,All Terrain Tactic...",10000.00,17.0,39.60,4


### 1.4 Find out manufactures of starships who could design a passenger capacity over 50
- Requirment: `1 request with HAVING`
- Explanation: By grouping the data by manufacturer, we could use HAVING to filter out manufacutres that has the capability to desgin starships with the average passenger capcity over 50.

In [86]:
pd.read_sql(
    '''

            
    ''', 
    conn
)

,manufacturer,average_passengers
0,"Hoersch-Kessel Drive, Inc.",139000
1,"Rendili StarDrive, Free Dac Volunteers Enginee...",48247
2,"Kuat Drive Yards, Fondor Shipyards",38000
3,Botajef Shipyards,30000
4,Rothana Heavy Engineering,16000
5,"Kuat Drive Yards, Allanteen Six shipyards",2000
6,Mon Calamari shipyards,1200
7,"Imperial Department of Military Research, Sien...",843
8,Corellian Engineering Corporation,207
9,"Gallofree Yards, Inc.",90


### 1.5 Find out the oldest person in the Star Wars
- Requirment: `1 request with CTE`
- Explaination: 
    - By creating a temporary table, we conver the string `19BBY`, 19 years before the battle of yavin, to 19. And we write another query to return the oldest person's name and age.
    - Before implementing queries, we have checked no one is born ABY.

In [106]:
pd.read_sql(
    '''

    ''', 
    conn
)

,aby_count
0,0


In [109]:
pd.read_sql(
    '''

    ''', 
    conn
)

,name,birth_year
0,Yoda,896
